In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re

<IPython.core.display.Javascript object>

In [3]:
# read the data
mds = pd.read_excel("F:\GoogleDrive\python\Installment_Size\mds.xlsx")

<IPython.core.display.Javascript object>

In [4]:
# mds.tail(1)

<IPython.core.display.Javascript object>

In [5]:
# Generate unique AC NO
mds["AC"] = mds["Branch ID"] + mds["Account Number"]

<IPython.core.display.Javascript object>

In [6]:
# Get the Column Names
mds.columns.values

array(['Branch ID', 'Account Number', 'Account Title', 'Product\nID',
       'Product Name', 'Rate', 'Current Balance', 'Open\n Date',
       'Maturity\nDate', 'Lien Marked', 'Lien Branch', 'Lien Account',
       'Lien Amount', 'Status', 'AC'], dtype=object)

<IPython.core.display.Javascript object>

In [7]:
# mds.columns=[re.sub('\n', ' ', w) for w in mds.columns.values]
# mds.columns=[re.sub(' +', ' ', w) for w in mds.columns.values]

<IPython.core.display.Javascript object>

In [8]:
# Clean the Column Headers
mds.columns = mds.columns.str.replace("\n", " ").str.replace(" +", " ")

<IPython.core.display.Javascript object>

In [9]:
mds.columns

Index(['Branch ID', 'Account Number', 'Account Title', 'Product ID',
       'Product Name', 'Rate', 'Current Balance', 'Open Date', 'Maturity Date',
       'Lien Marked', 'Lien Branch', 'Lien Account', 'Lien Amount', 'Status',
       'AC'],
      dtype='object')

<IPython.core.display.Javascript object>

In [10]:
# Get the Elapsed days for each AC
mds["Months"] = (
    pd.to_datetime("2019-09-30", format="%Y-%m-%d") - pd.to_datetime(mds["Open Date"])
).dt.days

<IPython.core.display.Javascript object>

In [11]:
# Filter the data based on Rate and get rid of unneccessary rows
mds = mds[
    (pd.notnull(mds["Product ID"]))
    & (mds["Rate"].isin([7.80, 8.72, 7.27, 8.08]))
    & (mds["Months"] >= 0)
]

<IPython.core.display.Javascript object>

In [12]:
# Get the number of months elapsed from the days
mds["Months"] = mds["Months"] / 30

<IPython.core.display.Javascript object>

In [13]:
# Round the fraction months
mds["Months"] = mds["Months"].apply(np.floor)

<IPython.core.display.Javascript object>

In [14]:
# Get the Installment Size
mds["Installment_Size"] = mds["Current Balance"] / mds["Months"]

<IPython.core.display.Javascript object>

In [15]:
# mds['Installment_Size']

<IPython.core.display.Javascript object>

In [16]:
# In case of the AC without even a month elapsed, the installment size is the current balance
pd.options.mode.use_inf_as_na = True
mds["Installment_Size"] = mds["Installment_Size"].fillna(mds["Current Balance"])

<IPython.core.display.Javascript object>

In [17]:
# Label the products using the Int Rate
product_dict = dict(
    {7.80: "OLD PRODUCT", 8.72: "OLD PRODUCT", 7.27: "NEW PRODUCT", 8.08: "NEW PRODUCT"}
)

<IPython.core.display.Javascript object>

In [18]:
mds["PRODUCT_TYPE"] = mds["Rate"].replace(product_dict)

<IPython.core.display.Javascript object>

In [19]:
# mds.head(1)

<IPython.core.display.Javascript object>

In [20]:
# Cut the fraction Instalment sizes into the actual Installment sizes
mds["ACTUAL_INSTALLMENT"] = pd.cut(
    mds["Installment_Size"],
    bins=[0, 1000, 2000, 5000, 10000, 15000, 20000, 25000, 30000, 100000000],
    labels=[500, 1000, 2000, 5000, 10000, 15000, 20000, 25000, 30000],
).astype("int64")

<IPython.core.display.Javascript object>

In [21]:
# mds.head(2)

<IPython.core.display.Javascript object>

In [22]:
# Get the Summary Table
mds.groupby(["PRODUCT_TYPE", "Rate"])["ACTUAL_INSTALLMENT"].sum()

PRODUCT_TYPE  Rate
NEW PRODUCT   7.27     4863500
              8.08     2154000
OLD PRODUCT   7.80    35507000
              8.72    17392500
Name: ACTUAL_INSTALLMENT, dtype: int64

<IPython.core.display.Javascript object>